In [11]:
# Set Testset Path
Testset_Path = 'Your Testset Path'

# Set pretrained model weights path
Model_Weights_Path = 'Your Model Weights Path' # Saved Model Weights Path


# Set number of chamber
num_CH = 4 # set this number of chamber as 2 or 4

In [12]:
#Install the library for U-Net Model (segmentation_models_pytorch)
!pip install -q -U segmentation-models-pytorch albumentations > /dev/null
import segmentation_models_pytorch as smp

In [13]:
!pip install SimpleITK
import SimpleITK as sitk


In [14]:
import numpy as np
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader

import torchvision.models
from torchvision import transforms
import torchvision.transforms.functional as tF

from skimage import io
import matplotlib.pyplot as plt
from PIL import Image

from pathlib import Path

import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [15]:
root_dir = Path(Testset_Path) # Testset Path
dir_list = os.listdir(root_dir)
dir_list.sort()
print(dir_list)

['.DS_Store', 'Test1', 'Test10', 'Test2', 'Test3', 'Test4', 'Test5', 'Test6', 'Test7', 'Test8', 'Test9', 'untitled.m']


In [18]:
# Load model and trained model weights
model = smp.Unet("resnet34", encoder_weights='imagenet', in_channels = 1, classes=1, activation='sigmoid')
model = model.to(device)
model = torch.load(Model_Weights_Path, map_location=torch.device('cpu'))

In [19]:
# Run on the testset
# Save the prediction sequence to .mhd/.raw file

for i in range(len(dir_list)):
    if 'Test' in dir_list[i]:
        
        # Run 2 Chamber view
        if num_CH == 2:
            seq_name = root_dir/dir_list[i]/Path(dir_list[i]+"_2CH_sequence.mhd")
            mdh_data = root_dir/dir_list[i]/Path("R_2CH_sequence.mhd")
            mdh_data = str(root_dir/dir_list[i]/Path("R_2CH_sequence"))
            raw_data = root_dir/dir_list[i]/Path("R_2CH_sequence.raw")


        # Run 4 Chamber view
        if num_CH == 4:
            seq_name = root_dir/dir_list[i]/Path(dir_list[i]+"_4CH_sequence.mhd")
            mdh_data = root_dir/dir_list[i]/Path("R_4CH_sequence.mhd")
            mdh_data = str(root_dir/dir_list[i]/Path("R_4CH_sequence"))
            raw_data = root_dir/dir_list[i]/Path("R_4CH_sequence.raw")
      
      # print(seq_name)
        print(mdh_data)
      # print(raw_data)
    
    
        seq = io.imread(seq_name)
        original_shape = seq.shape
        
        seq = np.transpose(seq,(1, 2, 0))

        transform = transforms.Compose(
          [
              transforms.ToTensor(),
              transforms.Resize((256, 256))
          ]
        )

        new_seq = transform(seq)

        output = np.zeros(original_shape)
        input = new_seq
        input = torch.unsqueeze(input, dim=1)
        input = input.to(device)

        out = model(input)
        out = (out>0.5).int()

        transform_back = transforms.Compose(
          [
              transforms.Resize((original_shape[1], original_shape[2]))
          ]
        )

        out = transform_back(out)
           
        
        for i in range(new_seq.shape[0]):
            output[i,:,:] = out[i].cpu().detach().permute(1, 2, 0)[:,:,0]
            
        # Create a SimpleITK image from the numpy array
        image = sitk.GetImageFromArray(output)

        # Set the image origin and spacing
        image.SetOrigin((0,0,0))
        image.SetSpacing((0.308, 0.154,  1.54))        
     
        # Set the data type of the image to be MET_CHAR
        image = sitk.Cast(image, sitk.sitkInt8)
        
        # Save the image in .mhd and .raw format
        sitk.WriteImage(image, mdh_data + ".mhd")

/Users/ychen215/Desktop/Project1/testset/Test1/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test10/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test2/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test3/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test4/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test5/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test6/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test7/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test8/R_4CH_sequence
/Users/ychen215/Desktop/Project1/testset/Test9/R_4CH_sequence
